In [ ]:
import pandas as pd
import re
import os
import matplotlib.pyplot as plt
from IPython.display import display
import warnings
warnings.simplefilter('always')

from scripts.a_data_loading_cleaning import run_load_clean_diagnose_data

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Data Loading & Cleaning

In [ ]:
csv_path = "results/scenarios_results_complete_tinyllama.csv"
df = run_load_clean_diagnose_data(csv_path)

✅ All rows have consistent 'total_generated_tokens' = 16384
----------------------------------------------------------------------------------------------------
⚠️ FLOPs values are NOT constant: [1.6949971e+13 0.0000000e+00]
Original distribution:
flops
0.000000e+00    1
1.694997e+13    9
Name: count, dtype: int64
- Dominant FLOPs value: 16949970993152.0
- Affected rows count: 1
- Affected row indices: [6]
- Affected configs: ['R2_Low_Latency_Chatbot_Deployment']
----------------------------------------------------------------------------------------------------
✅ Total generated tokens value is constant: 16384
Original distribution:
total_generated_tokens
16384    10
Name: count, dtype: int64
Identifying differentiators for FLOPs...
FLOP Differentiators:
fp_precision: {np.float64(16949970993152.0): 'torch.float16', np.float64(0.0): 'torch.float32'}
flops_per_token: {np.float64(16949970993152.0): np.float64(1034544128.0), np.float64(0.0): np.float64(0.0)}
------------------------------

/Users/henrybaker/repositories/thesis_analysis/scripts/a_data_loading_cleaning.py:522: UserWarning: ⚠️ FLOPs values are NOT constant: [1.6949971e+13 0.0000000e+00]
  verify_flops(df)
